In [1]:
print("My model of Machine Learning based on K nearest neightbour")

My model of Machine Learning based on K nearest neightbour


In [33]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display
import pandas as pd
import pymysql
from datetime import datetime, timedelta 
import math

In [73]:
def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums) - i - 1):  
            if nums[j][0] > nums[j + 1][0]:
                nums[j], nums[j + 1] = nums[j + 1], nums[j]

    return nums

In [74]:
def get_prediction_data(date,time):
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    weekday = datetime.strptime(date,"%Y-%m-%d").weekday()
    #day = datetime.strptime(date,"%Y-%m-%d").day
    time = str(time) + ":00:00"
    dic = get_forecast(date + ' ' + time)
    con = dic_w[dic['condition']]
    dic['condition'] = con
    dic['time'] = date + ' ' + time
    dic['weekday'] = weekday
    return dic

def get_forecast(date):
    day = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    day_now = datetime.now()
    day = (day - day_now).total_seconds()/3600
    hour = int(day / 3)
    url = 'https://api.openweathermap.org/data/2.5/forecast?lat=53.36&lon=-6.25&appid=dbb2b9eb4f9424b9c2c168ad52c077d9'
    info = requests.get(url).json()
    wind_speed = info['list'][hour]['wind']['speed']
    condition = info['list'][hour]['weather'][0]['main']
    temp = info['list'][hour]['main']['temp']
    return {'wind':wind_speed,'condition':condition,'temp':temp}

In [75]:
def get_csv():
    low = '2020-03-04 09:39:43'
    high = '2020-03-24 09:47:28'
    sql = "SELECT * FROM dbike.weather where `datetime` between '"+ low +"' and '"+ high +"';"
    conn = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    df = pd.read_sql(sql, con=conn)
    df.to_csv("old_weather.csv",index=False)

In [84]:
def get_old_weather(low,df):
    dic_old_weather = {}
    dic_w = {'Clear':1,'Clouds':2,'Mist':3,'Drizzle':4,'Fog':5,'Rain':6,'Snow':7}
    #low = datetime.strptime(low,"%Y-%m-%d %H:%M:%S")
    high = low + timedelta(hours = 8)
    #low = '2020-03-04 15:20:02'
    #high = '2020-03-04 15:40:02'
    temp=df[(df['datetime']>=low) & (df['datetime']<=high)]
    #temp.head(100)
    if temp.empty:
        return dic_old_weather
    temp.iloc[[0],[3]].values[0][0]
    #print(data)
    dic_old_weather['condition'] = dic_w[temp.iloc[[0],[3]].values[0][0]]
    dic_old_weather['wind'] = temp.iloc[[0],[9]].values[0][0]
    dic_old_weather['temp'] = temp.iloc[[0],[6]].values[0][0]
    return dic_old_weather

def prediction(number,date,time):
    #get_csv()
    db = pymysql.connect("dbike.cerj203fcxcq.eu-west-1.rds.amazonaws.com", "yuhao", "qwert2008", "dbike")
    cursor = db.cursor()
    sql="SELECT * FROM dbike.daily_data where `number` = " + str(number) + " group by last_update;"
    cursor.execute(sql)
    db.commit()
    data = cursor.fetchall()
    cursor.close()
    length = len(data)
    i = 0
    count = 0
    dic_pre = get_prediction_data(date,time)
    date_new = datetime.strptime(dic_pre['time'],"%Y-%m-%d %H:%M:%S")
    li = []
    print(length)
    df = pd.read_csv('old_weather.csv')
    df['datetime'] = df['datetime'].astype('datetime64')
    while i < length:
        date_old = data[i][4]
        available_bike_stands = data[i][2]
        available_bikes = data[i][3]
        weekday_old = date_old.weekday()
        dic_old = get_old_weather(date_old,df)
        i = i + 1
        if count == 5:
            break
        if not dic_old:
            continue
        d_condition = float(dic_pre['condition'])-float(dic_old['condition'])
        d_wind = (float(dic_pre['wind']) - float(dic_old['wind']))*5
        if date_new.hour*10000+date_new.minute*100+date_new.second > date_old.hour*10000+date_old.minute*100+date_old.second:
            d_time = (date_new - date_old).seconds/3600
        else:
            d_time = (date_old - date_new).seconds/3600
        d_week = int(dic_pre['weekday'])-int(weekday_old)
        d_time = round(d_time,3)
        #print(d_condition)
        #print(d_wind)
        print(d_time,end=" ")
        #print(d_week)
        distance = math.sqrt(d_wind**2 + d_condition**2 + d_time**2 + d_week**2)
        li.append([distance,available_bikes,available_bike_stands])
        #print(li)
    print(len(li))
    bubble_sort(li)
    print(li[0],li[1],li[2])
    weight = 1/li[0][0] + 1/li[1][0] + 1/li[2][0] + 1/li[3][0] + 1/li[4][0]
    prediction_bikes = (int(li[0][1])*(1/li[0][0]) + int(li[1][1])*(1/li[1][0]) + int(li[2][1])*(1/li[2][0]) + int(li[3][1])*(1/li[3][0]) + int(li[4][1])*(1/li[4][0]))/ weight
    prediction_stands = (int(li[0][2])*(1/li[0][0]) + int(li[1][2])*(1/li[1][0]) + int(li[2][2])*(1/li[2][0]) + int(li[3][2])*(1/li[3][0]) + int(li[4][2])*(1/li[4][0]))/weight
    dic = {}
    dic_w = {'1':'Clear','2':'Clouds','3':'Mist','4':'Drizzle','5':'Fog','6':'Rain','7':'Snow'}
    dic['prediction_bikes']=prediction_bikes
    dic['prediction_stands']=prediction_stands
    dic['condition'] = dic_w[str(dic_pre['condition'])]
    dic['wind'] = dic_pre['wind']
    dic['temp'] = dic_pre['temp']
    return dic

In [85]:
print(prediction(32,'2020-04-12','09'))

3405


0.662 0.679 1.501 5.844 5.899 5.905 5.982 6.128 6.195 6.302 6.46 6.662 6.831 6.838 6.96 7.082 7.144 7.249 7.329 7.426 7.581 7.665 7.703 7.832 7.908 7.999 8.076 8.157 8.327 8.402 8.493 8.549 8.648 8.737 8.813 8.896 8.992 9.057 9.088 9.223 9.328 9.402 9.498 9.577 9.651 9.746 9.785 9.889 9.981 10.056 10.112 10.209 10.408 10.468 10.582 10.657 10.739 10.812 10.906 10.967 11.069 11.108 11.207 11.283 11.397 11.566 11.653 11.744 11.779 11.948 12.07 12.239 12.382 12.491 12.63 12.803 12.99 13.158 13.168 13.321 13.347 13.516 13.61 13.779 13.949 14.118 14.287 14.456 14.626 14.795 14.948 8.883 8.814 8.645 8.448 8.279 8.11 7.94 7.771 7.602 7.433 7.264 7.094 6.925 6.756 6.587 6.418 6.248 6.079 5.918 5.88 5.711 5.542 5.373 5.204 5.035 4.866 4.697 4.528 4.359 4.19 4.021 3.985 3.773 3.621 3.453 3.303 3.174 3.006 2.837 2.667 2.627 2.552 2.383 2.322 2.104 2.002 1.939 1.83 1.61 1.483 1.349 1.308 1.184 1.091 1.017 0.946 0.892 0.762 0.683 0.59 0.513 0.423 0.34 0.258 0.084 0.003 0.073 0.142 0.234 0.297 0.412 

5.979 6.006 6.157 6.247 6.303 6.415 6.452 6.581 6.646 6.744 6.791 6.891 6.935 7.027 7.16 7.247 7.32 7.495 7.551 7.635 7.745 7.824 7.838 7.96 8.036 8.141 8.221 8.327 8.414 8.499 8.574 8.659 8.74 8.826 8.87 8.981 9.073 9.166 9.178 9.285 9.348 9.476 9.586 9.829 9.867 10.061 10.166 10.227 10.321 10.484 10.57 10.662 10.731 10.819 10.859 10.957 11.076 11.158 11.249 11.281 11.371 11.532 11.74 11.909 12.071 12.24 12.409 12.474 12.52 12.689 12.859 12.963 13.132 13.332 13.416 13.444 13.578 13.661 13.811 13.917 14.121 14.213 14.381 14.55 14.719 14.887 8.944 8.775 8.606 8.457 8.288 8.119 7.95 7.779 7.61 7.441 7.272 7.103 6.934 6.765 6.596 6.427 6.258 6.089 5.918 5.861 5.691 5.522 5.353 5.184 5.016 4.846 4.678 4.508 4.339 4.17 4.001 3.993 3.824 3.731 3.535 3.366 3.269 3.163 2.994 2.825 2.704 2.585 2.569 2.4 2.311 2.174 2.028 1.938 1.869 1.823 1.676 1.613 1.517 1.425 1.253 1.176 1.162 1.008 0.917 0.877 0.787 0.673 0.641 0.524 0.489 0.376 0.259 0.183 0.111 0.002 0.046 0.086 0.239 0.277 0.339 0.499 0.

2.829 2.661 2.492 2.351 2.182 2.013 1.844 1.69 1.636 1.579 1.411 1.266 1.097 1.025 0.941 0.771 0.624 0.455 0.282 0.139 0.066 0.103 0.199 0.394 0.469 0.638 0.807 0.976 1.151 1.168 1.3 1.469 1.638 1.807 1.976 2.145 2.314 2.482 2.651 2.662 2.831 3.0 3.114 3.283 3.452 3.621 3.741 3.982 4.022 4.191 4.36 4.441 4.61 4.761 4.93 5.099 5.206 5.375 5.544 5.713 5.882 6.051 6.22 6.399 6.568 6.607 6.718 6.888 6.915 7.003 7.119 7.244 7.413 7.582 7.758 7.927 8.096 8.292 8.427 8.535 8.704 8.891 8.904 9.082 9.251 9.347 9.548 9.717 9.886 9.933 10.102 10.271 10.33 10.525 10.694 10.863 11.009 11.178 11.287 11.456 11.625 11.794 11.963 12.132 12.3 12.334 12.502 12.64 12.654 12.823 13.027 13.196 13.365 13.534 13.703 13.872 14.041 14.211 14.379 14.548 14.717 14.886 8.945 8.776 8.607 8.468 8.299 8.13 7.961 7.792 7.623 7.454 7.285 7.116 6.947 6.777 6.608 6.439 6.271 6.102 5.962 5.861 5.692 5.523 5.353 5.184 5.015 4.846 4.677 4.508 4.339 4.169 3.994 3.824 3.655 3.486 3.317 3.147 2.978 2.809 2.64 2.471 2.301 2.132

In [83]:
#temp.head()
#get_csv()

In [72]:
#b=[[12.34,3,4],[12.54,4,5],[12.354,5,4],[13.4564,7,3],[11.342,3,2],[10.34,3,4],[19.54,4,5],[2.354,5,4],[3.4564,7,3],[1.342,3,2]]
#print(bubble_sort(b))

[[1.342, 3, 2], [2.354, 5, 4], [3.4564, 7, 3], [10.34, 3, 4], [11.342, 3, 2], [12.34, 3, 4], [12.354, 5, 4], [12.54, 4, 5], [13.4564, 7, 3], [19.54, 4, 5]]


In [ ]:
'''
t1 = '2020-03-04 21:18:29'
t2 = '2020-03-06 23:25:59'
t1 = datetime.strptime(t1,"%Y-%m-%d %H:%M:%S")
t2 = datetime.strptime(t2,"%Y-%m-%d %H:%M:%S")
t = t2-t1
t = t.seconds/1800
print(t)
t = ((t2-t1).total_seconds())/(3600*24)
t2 = '2020-04-11'
t2 = datetime.strptime(t2,"%Y-%m-%d").day
t = datetime.now().day
print(t)
date = '2020-04-14 21:18:29'
day = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
day_now = datetime.now()
day = (day - day_now).total_seconds()/3600
print(day/3)
'''